In [265]:
import requests
import json
from pprint import pprint
import numpy as np
import pandas as pd
import os
import csv
import datetime as dt

import sqlalchemy
from sqlalchemy import create_engine, func, inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date
from opencage.geocoder import OpenCageGeocode # used to get long and lat of given city name
from math import radians, sin, cos, asin, sqrt # used to calculate distance of room and city longs and lats

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session


from flask import Flask, jsonify

In [190]:
results = requests.get('https://public.opendatasoft.com/api/records/1.0/download/?dataset=air-bnb-listings&rows=100&format=json').json()


In [191]:
pprint(results)

[{'datasetid': 'air-bnb-listings',
  'fields': {'availability_365': 347,
             'calculated_host_listings_count': 74,
             'city': 'New-orleans',
             'column_10': 1003,
             'column_19': 'United states',
             'column_20': 'United states, New-orleans, Central Business '
                          'District',
             'coordinates': [29.952860923402337, -90.07131139281506],
             'host_id': 114353388,
             'id': 19051656,
             'last_review': '2017-10-19',
             'minimum_nights': 1,
             'name': 'Gorgeous Stay Alfred on North Carondelet Street',
             'neighbourhood': 'Central Business District',
             'number_of_reviews': 1,
             'reviews_per_month': 0.05,
             'room_type': 'Entire home/apt',
             'updated_date': '2019-05-05'},
  'geometry': {'coordinates': [-90.07131139281506, 29.952860923402337],
               'type': 'Point'},
  'record_timestamp': '2019-06-07T06:01:0

             'number_of_reviews': 8,
             'reviews_per_month': 0.51,
             'room_type': 'Entire home/apt',
             'updated_date': '2019-05-05'},
  'geometry': {'coordinates': [-90.06790251854413, 29.927262108077237],
               'type': 'Point'},
  'record_timestamp': '2019-06-07T06:01:00Z',
  'recordid': '62541dd4554d692377565c3e6dbedb891650c29f'},
 {'datasetid': 'air-bnb-listings',
  'fields': {'availability_365': 364,
             'calculated_host_listings_count': 4,
             'city': 'New-orleans',
             'column_10': 95,
             'column_19': 'United states',
             'column_20': 'United states, New-orleans, Pines Village',
             'coordinates': [30.02602247811368, -90.01963939114829],
             'host_id': 127084738,
             'id': 22639065,
             'last_review': '2019-04-01',
             'minimum_nights': 1,
             'name': 'Upstairs Retreat',
             'neighbourhood': 'Pines Village',
             'number_of_

             'host_id': 48187766,
             'id': 24582536,
             'last_review': '2019-04-20',
             'minimum_nights': 1,
             'name': '2furnished room/private entrance/Tulane University',
             'neighbourhood': 'East Carrollton',
             'number_of_reviews': 32,
             'reviews_per_month': 2.57,
             'room_type': 'Private room',
             'updated_date': '2019-05-05'},
  'geometry': {'coordinates': [-90.1261717398774, 29.94247477570772],
               'type': 'Point'},
  'record_timestamp': '2019-06-07T06:01:00Z',
  'recordid': 'be759894e0b44df6068f951749e4c849a72882c1'},
 {'datasetid': 'air-bnb-listings',
  'fields': {'availability_365': 72,
             'calculated_host_listings_count': 6,
             'city': 'New-orleans',
             'column_10': 120,
             'column_19': 'United states',
             'column_20': 'United states, New-orleans, French Quarter',
             'coordinates': [29.95164959814229, -90.065799234

             'calculated_host_listings_count': 5,
             'city': 'New-orleans',
             'column_10': 499,
             'column_19': 'United states',
             'column_20': 'United states, New-orleans, Lower Garden District',
             'coordinates': [29.930799079858346, -90.07509737599439],
             'host_id': 37614659,
             'id': 29234907,
             'last_review': '2019-04-07',
             'minimum_nights': 3,
             'name': 'Casabianca: Elegant Flat in Historic Magazine Row',
             'neighbourhood': 'Lower Garden District',
             'number_of_reviews': 15,
             'reviews_per_month': 2.74,
             'room_type': 'Entire home/apt',
             'updated_date': '2019-05-05'},
  'geometry': {'coordinates': [-90.07509737599439, 29.930799079858346],
               'type': 'Point'},
  'record_timestamp': '2019-06-07T06:01:00Z',
  'recordid': 'ae1b4133d6de9b32fe1b0d36014a08ef36f669c3'},
 {'datasetid': 'air-bnb-listings',
  'fields':

In [192]:
results = [r["fields"] for r in results]
results

[{'updated_date': '2019-05-05',
  'reviews_per_month': 0.05,
  'calculated_host_listings_count': 74,
  'city': 'New-orleans',
  'minimum_nights': 1,
  'coordinates': [29.952860923402337, -90.07131139281506],
  'room_type': 'Entire home/apt',
  'number_of_reviews': 1,
  'column_10': 1003,
  'neighbourhood': 'Central Business District',
  'last_review': '2017-10-19',
  'host_id': 114353388,
  'column_19': 'United states',
  'availability_365': 347,
  'id': 19051656,
  'column_20': 'United states, New-orleans, Central Business District',
  'name': 'Gorgeous Stay Alfred on North Carondelet Street'},
 {'updated_date': '2019-05-05',
  'reviews_per_month': 4.84,
  'calculated_host_listings_count': 1,
  'city': 'New-orleans',
  'minimum_nights': 1,
  'coordinates': [29.987034337344596, -89.94496848089439],
  'room_type': 'Entire home/apt',
  'number_of_reviews': 20,
  'column_10': 74,
  'neighbourhood': 'Viavant - Venetian Isles',
  'last_review': '2019-04-28',
  'host_id': 61310992,
  'column

In [193]:
results_df = pd.DataFrame(results)
results_df.head()

,availability_365,calculated_host_listings_count,city,column_10,column_19,column_20,coordinates,host_id,id,last_review,minimum_nights,name,neighbourhood,number_of_reviews,reviews_per_month,room_type,updated_date
0,347,74,New-orleans,1003,United states,"United states, New-orleans, Central Business D...","[29.952860923402337, -90.07131139281506]",114353388,19051656,2017-10-19,1,Gorgeous Stay Alfred on North Carondelet Street,Central Business District,1,0.05,Entire home/apt,2019-05-05
1,302,1,New-orleans,74,United states,"United states, New-orleans, Viavant - Venetian...","[29.987034337344596, -89.94496848089439]",61310992,19079967,2019-04-28,1,Cozy New Orleans Efficiency On The Water,Viavant - Venetian Isles,20,4.84,Entire home/apt,2019-05-05
2,206,1,New-orleans,175,United states,"United states, New-orleans, Marlyville - Fonta...","[29.954771004938046, -90.11994216301255]",22171971,19123547,2019-04-22,2,Townhome on the Streetcar Line,Marlyville - Fontainbleau,74,3.65,Entire home/apt,2019-05-05
3,0,2,New-orleans,86,United states,"United states, New-orleans, Lower Garden District","[29.93808917158898, -90.07359074429189]",710534,19245713,2019-04-29,2,Stylish 1BR Shotgun in Lower Garden District,Lower Garden District,89,3.90,Entire home/apt,2019-05-05
4,25,4,New-orleans,84,United states,"United states, New-orleans, Seventh Ward","[29.971855482779855, -90.06922180405711]",2428040,19267950,2019-04-27,1,Relaxing 3BR Balcony Apartment,Seventh Ward,22,1.08,Entire home/apt,2019-05-05


In [194]:
cleanData = results_df[[ 'column_10','id', 'coordinates', 'room_type', "city", 'column_19']]
cleanData.head()

,column_10,id,coordinates,room_type,city,column_19
0,1003,19051656,"[29.952860923402337, -90.07131139281506]",Entire home/apt,New-orleans,United states
1,74,19079967,"[29.987034337344596, -89.94496848089439]",Entire home/apt,New-orleans,United states
2,175,19123547,"[29.954771004938046, -90.11994216301255]",Entire home/apt,New-orleans,United states
3,86,19245713,"[29.93808917158898, -90.07359074429189]",Entire home/apt,New-orleans,United states
4,84,19267950,"[29.971855482779855, -90.06922180405711]",Entire home/apt,New-orleans,United states


In [195]:
cleanData = cleanData.rename(index=str, columns={"column_10":"Price",
                                                 "id":"Room_Id", 
                                                 "coordinates":"Coordinates",
                                                 "column_19":"Country", 
                                                 "room_type":"Room_Type",
                                                 "city":"City"})

cleanData.head()

,Price,Room_Id,Coordinates,Room_Type,City,Country
0,1003,19051656,"[29.952860923402337, -90.07131139281506]",Entire home/apt,New-orleans,United states
1,74,19079967,"[29.987034337344596, -89.94496848089439]",Entire home/apt,New-orleans,United states
2,175,19123547,"[29.954771004938046, -90.11994216301255]",Entire home/apt,New-orleans,United states
3,86,19245713,"[29.93808917158898, -90.07359074429189]",Entire home/apt,New-orleans,United states
4,84,19267950,"[29.971855482779855, -90.06922180405711]",Entire home/apt,New-orleans,United states


In [196]:
# Create two lists for the loop results to be placed
lat = []
lon = []

# For each row in a varible,
for row in cleanData['Coordinates']:
   # Try to,
   try:
       # Split the row by comma and append
       # everything before the comma to lat
       lat.append(row[0])
       # Split the row by comma and append
       # everything after the comma to lon
       lon.append(row[1])
   # But if you get an error
   except:
       # append a missing value to lat
       lat.append(np.NaN)
       # append a missing value to lon
       lon.append(np.NaN)

# Create two new columns from lat and lon
cleanData['Room_Lat'] = lat
cleanData['Room_Long'] = lon



In [197]:
cleanData.drop(columns=["Coordinates"])
cleanData.head()

,Price,Room_Id,Coordinates,Room_Type,City,Country,Room_Lat,Room_Long
0,1003,19051656,"[29.952860923402337, -90.07131139281506]",Entire home/apt,New-orleans,United states,29.952861,-90.071311
1,74,19079967,"[29.987034337344596, -89.94496848089439]",Entire home/apt,New-orleans,United states,29.987034,-89.944968
2,175,19123547,"[29.954771004938046, -90.11994216301255]",Entire home/apt,New-orleans,United states,29.954771,-90.119942
3,86,19245713,"[29.93808917158898, -90.07359074429189]",Entire home/apt,New-orleans,United states,29.938089,-90.073591
4,84,19267950,"[29.971855482779855, -90.06922180405711]",Entire home/apt,New-orleans,United states,29.971855,-90.069222


In [198]:
cleanData["Selection"] = cleanData["City"]+", "+cleanData["Country"]
cleanData.head(5)

,Price,Room_Id,Coordinates,Room_Type,City,Country,Room_Lat,Room_Long,Selection
0,1003,19051656,"[29.952860923402337, -90.07131139281506]",Entire home/apt,New-orleans,United states,29.952861,-90.071311,"New-orleans, United states"
1,74,19079967,"[29.987034337344596, -89.94496848089439]",Entire home/apt,New-orleans,United states,29.987034,-89.944968,"New-orleans, United states"
2,175,19123547,"[29.954771004938046, -90.11994216301255]",Entire home/apt,New-orleans,United states,29.954771,-90.119942,"New-orleans, United states"
3,86,19245713,"[29.93808917158898, -90.07359074429189]",Entire home/apt,New-orleans,United states,29.938089,-90.073591,"New-orleans, United states"
4,84,19267950,"[29.971855482779855, -90.06922180405711]",Entire home/apt,New-orleans,United states,29.971855,-90.069222,"New-orleans, United states"


In [199]:
cleanData['City'] = cleanData['City'].str.replace('-',' ')
cleanData.head()

,Price,Room_Id,Coordinates,Room_Type,City,Country,Room_Lat,Room_Long,Selection
0,1003,19051656,"[29.952860923402337, -90.07131139281506]",Entire home/apt,New orleans,United states,29.952861,-90.071311,"New-orleans, United states"
1,74,19079967,"[29.987034337344596, -89.94496848089439]",Entire home/apt,New orleans,United states,29.987034,-89.944968,"New-orleans, United states"
2,175,19123547,"[29.954771004938046, -90.11994216301255]",Entire home/apt,New orleans,United states,29.954771,-90.119942,"New-orleans, United states"
3,86,19245713,"[29.93808917158898, -90.07359074429189]",Entire home/apt,New orleans,United states,29.938089,-90.073591,"New-orleans, United states"
4,84,19267950,"[29.971855482779855, -90.06922180405711]",Entire home/apt,New orleans,United states,29.971855,-90.069222,"New-orleans, United states"


In [200]:
# cleanData.City.unique()

In [301]:
cities_df = cleanData[['City']]
cities_df.head(10)

,City
0,New orleans
1,New orleans
2,New orleans
3,New orleans
4,New orleans
5,New orleans
6,New orleans
7,New orleans
8,New orleans
9,New orleans


In [303]:
#Export city names to csv
cities_df.to_csv('../cities.csv', index=False)

In [324]:
# Get data for each city in cities.csv and call API
cities_long_lat = []
with open("../cities.csv") as cities_file:
    cities_reader = csv.reader(cities_file)
    cities = [city[0] for city in cities_reader]
    cities_long_lat = [ow.get_current(city, **settings) for city in cities] # API call to get city long lat from city name

In [325]:
# Create an "extracts" object to get the temperature, latitude,
# and longitude in each city
summary = ["main.temp", "coord.lat", "coord.lon"]

# Create a Pandas DataFrame with the results
data = [response(*summary) for response in cities_long_lat]

cities_long_lat = pd.DataFrame(data, index=cities)
cities_long_lat.head()

,0,1,2
City,4.49,-35.28,149.13
New orleans,33.10,29.95,-90.07
New orleans,33.10,29.95,-90.07
New orleans,33.10,29.95,-90.07
New orleans,33.10,29.95,-90.07


## working code below

In [322]:
rooms_long_lat= cleanData[['City','Room_Lat','Room_Long']]

rooms_long_lat["Room_Lat"] = rooms_long_lat["Room_Lat"].map("{:.2f}".format)
rooms_long_lat["Room_Long"] = rooms_long_lat["Room_Long"].map("{:,.2f}".format)

rooms_long_lat.head()

/anaconda2/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda2/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,City,Room_Lat,Room_Long
0,New orleans,29.95,-90.07
1,New orleans,29.99,-89.94
2,New orleans,29.95,-90.12
3,New orleans,29.94,-90.07
4,New orleans,29.97,-90.07


In [319]:
combined_long_lat_df = pd.merge(cities_long_lat, rooms_long_lat, on='City', how='outer')

KeyError: 'City'

In [313]:
combined_long_lat_df

NameError: name 'combined_long_lat_df' is not defined

In [115]:
# this site confirmed the calculation is correct:
# https://www.gpsvisualizer.com/calculators

# OpenCageGeocode API call to get long lat of city name
API_key = "30c5a732512c41af8f052a21f806fcce"
geocoder = OpenCageGeocode(API_key)

rooms_lat = []   # create empty lists
rooms_long = []
cities_lat = [] 
cities_long = []
calculated_distance = []

for index, row in cleanData.iterrows():
    room_lat1 = row['Room_Lat']
    room_long1 = row['Room_Long']
    
    results = geocoder.geocode(row['Selection'])
    city_lat2 = results[0]['geometry']['lat']
    city_long2 = results[0]['geometry']['lng']
    
    def distance(room_long1, room_lat1, city_long2, city_lat2):
        room_long1, room_lat1, city_long2, city_lat2 = map(radians, [room_long1, room_lat1, city_long2, city_lat2])
        dlon = city_long2 - room_long1
        dlat = city_lat2 - room_lat1
        a = sin(dlat / 2) ** 2 + cos(room_lat1) * cos(city_lat2) * sin(dlon / 2) ** 2
        return 2 * 3958.756 * asin(sqrt(a)) #6371 km / 3958.756 for miles

    room_city_distance = distance(room_long1, room_lat1, city_long2, city_lat2)

    
    calculated_distance.append(room_city_distance)
    rooms_lat.append(room_lat)
    rooms_long.append(room_long)
    cities_lat.append(city_lat)
    cities_long.append(city_long)
    
cleanData['City_Lat'] = cities_lat
cleanData['City_Long'] = cities_long
cleanData['Calc_Distance'] = calculated_distance


In [161]:
# cleanData.head()

In [163]:
# cleanData.City.unique()

In [53]:
# conn_str = "root:***password***@localhost/airbnb_data"
# engine = create_engine(f'mysql://{conn_str}')

# #check for tables in database
# #engine.table_names()

# #enter data into database
# ***df***.to_sql(name="airbnb_data_tbl", con=engine, if_exists="append",index=False)

In [ ]:

# # OpenCageGeocode API call to get long lat of city name
# API_key = "30c5a732512c41af8f052a21f806fcce"
# geocoder = OpenCageGeocode(API_key)

# rooms_lat = []   # create empty lists
# rooms_long = []
# cities_lat = [] 
# cities_long = []
# calculated_distance = []

# for index, row in cleanData.iterrows():
#     room_lat1 = row['Latitude']
#     room_long1 = row['Longitude']
    
#     results = geocoder.geocode(row['Selection'])
#     city_lat2 = results[0]['geometry']['lat']
#     city_long2 = results[0]['geometry']['lng']
    
#     def distance(room_long1, room_lat1, city_long2, city_lat2):
#         room_long1, room_lat1, city_long2, city_lat2 = map(radians, [room_long1, room_lat1, city_long2, city_lat2])
#         dlon = city_long2 - room_long1
#         dlat = city_lat2 - room_lat1
#         a = sin(dlat / 2) ** 2 + cos(room_lat1) * cos(city_lat2) * sin(dlon / 2) ** 2
#         return 2 * 3958.756 * asin(sqrt(a)) #6371 km / 3958.756 for miles

#     room_city_distance = distance(room_long1, room_lat1, city_long2, city_lat2)
    
#     calculated_distance.append(room_city_distance)
#     rooms_lat.append(room_lat)
#     rooms_long.append(room_long)
#     cities_lat.append(city_lat)
#     cities_long.append(city_long)

In [81]:
# print("rooms_lat:", rooms_lat)
# print("rooms_long:", rooms_long)
# print("cities_lat:", cities_lat)
# print("cities_long:", cities_long)
# print("calculated_distance:", calculated_distance)

In [73]:
# # OpenCageGeocode API call to get long lat of city name
# API_key = "30c5a732512c41af8f052a21f806fcce"
# geocoder = OpenCageGeocode(API_key)

# rooms_lat = []   # create empty lists
# rooms_long = []
# cities_lat = [] 
# cities_long = []

# for index, row in cleanData.iterrows():
#     room_lat = row['Latitude']
#     room_long = row['Longitude']
    
#     results = geocoder.geocode(row['Selection'])
#     city_lat = results[0]['geometry']['lat']
#     city_long = results[0]['geometry']['lng']
    
#     rooms_lat.append(room_lat)
#     rooms_long.append(room_long)
#     cities_lat.append(city_lat)
#     cities_long.append(city_long)


In [229]:
# Dependencies
# import openweathermapy.core as owm
import openweathermapy as ow

api_key = "3567360c8a78f247cb21275118a6ae30"
#config
# from config import api_key

In [230]:
# Create settings dictionary with information we're interested in
settings = {"units": "metric", "appid": api_key}

In [248]:
city_data = []
for index, row in cleanData.iterrows():
    city_info = [ow.get_current(row["City"], **settings)]
    city_data.append(city_info)

In [249]:
city_data

[[{'coord': {'lon': -90.07, 'lat': 29.95},
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03d'}],
   'base': 'stations',
   'main': {'temp': 33.54,
    'pressure': 1018,
    'humidity': 55,
    'temp_min': 31.67,
    'temp_max': 35},
   'visibility': 12874,
   'wind': {'speed': 5.1, 'deg': 210},
   'clouds': {'all': 40},
   'dt': 1561238350,
   'sys': {'type': 1,
    'id': 5067,
    'message': 0.0097,
    'country': 'US',
    'sunrise': 1561201201,
    'sunset': 1561251864},
   'timezone': -18000,
   'id': 4335045,
   'name': 'New Orleans',
   'cod': 200}],
 [{'coord': {'lon': -90.07, 'lat': 29.95},
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03d'}],
   'base': 'stations',
   'main': {'temp': 33.54,
    'pressure': 1018,
    'humidity': 55,
    'temp_min': 31.67,
    'temp_max': 35},
   'visibility': 12874,
   'wind': {'speed': 5.1, 'deg': 210},
   'clouds': {'all': 40},

In [243]:
cities_name = []
cities_lat = [] 
cities_long = []

for index, row in cleanData.iterrows():
    
    results = [ow.get_current(row, **settings)]
    city = results[0]["name"]
#     city_lat = results[0]["coord.lat"]
#     city_long = results[0]["coord.lon"]

    cities_name.append(city)
#     cities_lat.append(city_lat)
#     cities_long.append(city_long)

print(city)
# print(city_long)
# print(city_lat)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
# Get data for each city in cities.csv
weather_data = []
with open("../Resources/cities.csv") as cities_file:
    cities_reader = csv.reader(cities_file)
#     cities = [city[0] for city in cities_reader]
    weather_data = [ow.get_current(city, **settings) for city in cities]

# Create an "extracts" object to get the temperature, latitude,
# and longitude in each city
summary = ["main.temp", "coord.lat", "coord.lon"]

# Create a Pandas DataFrame with the results
data = [response(*summary) for response in weather_data]

weather_data = pd.DataFrame(data, index=cities)
weather_data

city = ["name"]
city_long = ["coord.lon"]
city_lat = ["coord.lat"]

city = get_city_data(*city)
city_long = get_city_data(*city_long)
city_lat = get_city_data(*city_lat)
# print(f"The current weather summary for Paris is: {data}.")

print(city)
print(city_long)
print(city_lat)

In [220]:
City_Data = []
# romms_cities = []
# rooms_lat = []   # create empty lists
# rooms_long = []

#     results = geocoder.geocode(row['Selection'])
#     city_lat2 = results[0]['geometry']['lat']
#     city_long2 = results[0]['geometry']['lng']

# #Loops through each room long/lat and pushes to list
for index, row in cityData.iterrows():
#     room_city = row['City']
    city_info = [ow.get_current("City", **settings)]
    City_Data.append(city_info)
    
    # Create an "extracts" object to get the temperature, latitude,
# and longitude in each city
summary = ["main.temp", "coord.lat", "coord.lon"]

# city = ["name"]
city_long = ["coord.lon"]
city_lat = ["coord.lat"]

# Create a Pandas DataFrame with the results
data = [response(*city_long) for response in city_data]

weather_data = pd.DataFrame(data, index=cities)
weather_data

#     city = ["name"]
# city_long = ["coord.lon"]
# city_lat = ["coord.lat"]

# city = get_city_data(*city)
# city_long = get_city_data(*city_long)
# city_lat = get_city_data(*city_lat)
# # print(f"The current weather summary for Paris is: {data}.")

# print(city)
# print(city_long)
# print(city_lat)
    
#     room_city = row['City']
#     room_lat = row['Latitude']
#     room_long = row['Longitude']
      
      #rooms_cities.append(room_city)
#     rooms_lat.append(room_lat)
#     rooms_long.append(room_long)

    
    print(city_data)
# # print(rooms_long)

[{'coord': {'lon': 149.13, 'lat': -35.28}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 1.13, 'pressure': 1027, 'humidity': 100, 'temp_min': -3.33, 'temp_max': 4.44}, 'visibility': 10000, 'wind': {'speed': 2.41, 'deg': 193.587}, 'clouds': {'all': 28}, 'dt': 1561235709, 'sys': {'type': 1, 'id': 9588, 'message': 0.0078, 'country': 'AU', 'sunrise': 1561237936, 'sunset': 1561273121}, 'timezone': 36000, 'id': 2172517, 'name': 'City', 'cod': 200}]
[{'coord': {'lon': 149.13, 'lat': -35.28}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 1.13, 'pressure': 1027, 'humidity': 100, 'temp_min': -3.33, 'temp_max': 4.44}, 'visibility': 10000, 'wind': {'speed': 2.41, 'deg': 193.587}, 'clouds': {'all': 28}, 'dt': 1561235709, 'sys': {'type': 1, 'id': 9588, 'message': 0.0078, 'country': 'AU', 'sunrise': 1561237936, 'sunset': 1561273121},

[{'coord': {'lon': 149.13, 'lat': -35.28}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 1.13, 'pressure': 1027, 'humidity': 100, 'temp_min': -3.33, 'temp_max': 4.44}, 'visibility': 10000, 'wind': {'speed': 2.41, 'deg': 193.587}, 'clouds': {'all': 28}, 'dt': 1561235709, 'sys': {'type': 1, 'id': 9588, 'message': 0.0078, 'country': 'AU', 'sunrise': 1561237936, 'sunset': 1561273121}, 'timezone': 36000, 'id': 2172517, 'name': 'City', 'cod': 200}]
[{'coord': {'lon': 149.13, 'lat': -35.28}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 1.13, 'pressure': 1027, 'humidity': 100, 'temp_min': -3.33, 'temp_max': 4.44}, 'visibility': 10000, 'wind': {'speed': 2.41, 'deg': 193.587}, 'clouds': {'all': 28}, 'dt': 1561235709, 'sys': {'type': 1, 'id': 9588, 'message': 0.0078, 'country': 'AU', 'sunrise': 1561237936, 'sunset': 1561273121},

[{'coord': {'lon': 149.13, 'lat': -35.28}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 1.13, 'pressure': 1027, 'humidity': 100, 'temp_min': -3.33, 'temp_max': 4.44}, 'visibility': 10000, 'wind': {'speed': 2.41, 'deg': 193.587}, 'clouds': {'all': 28}, 'dt': 1561235708, 'sys': {'type': 1, 'id': 9588, 'message': 0.0073, 'country': 'AU', 'sunrise': 1561237936, 'sunset': 1561273121}, 'timezone': 36000, 'id': 2172517, 'name': 'City', 'cod': 200}]
[{'coord': {'lon': 149.13, 'lat': -35.28}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 1.13, 'pressure': 1027, 'humidity': 100, 'temp_min': -3.33, 'temp_max': 4.44}, 'visibility': 10000, 'wind': {'speed': 2.41, 'deg': 193.587}, 'clouds': {'all': 28}, 'dt': 1561235708, 'sys': {'type': 1, 'id': 9588, 'message': 0.0073, 'country': 'AU', 'sunrise': 1561237936, 'sunset': 1561273121},

[{'coord': {'lon': 149.13, 'lat': -35.28}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 1.13, 'pressure': 1027, 'humidity': 100, 'temp_min': -3.33, 'temp_max': 4.44}, 'visibility': 10000, 'wind': {'speed': 2.41, 'deg': 193.587}, 'clouds': {'all': 28}, 'dt': 1561235708, 'sys': {'type': 1, 'id': 9588, 'message': 0.0073, 'country': 'AU', 'sunrise': 1561237936, 'sunset': 1561273121}, 'timezone': 36000, 'id': 2172517, 'name': 'City', 'cod': 200}]
[{'coord': {'lon': 149.13, 'lat': -35.28}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 1.13, 'pressure': 1027, 'humidity': 100, 'temp_min': -3.33, 'temp_max': 4.44}, 'visibility': 10000, 'wind': {'speed': 2.41, 'deg': 193.587}, 'clouds': {'all': 28}, 'dt': 1561235708, 'sys': {'type': 1, 'id': 9588, 'message': 0.0073, 'country': 'AU', 'sunrise': 1561237936, 'sunset': 1561273121},

[{'coord': {'lon': 149.13, 'lat': -35.28}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 1.13, 'pressure': 1027, 'humidity': 100, 'temp_min': -3.33, 'temp_max': 4.44}, 'visibility': 10000, 'wind': {'speed': 2.41, 'deg': 193.587}, 'clouds': {'all': 28}, 'dt': 1561235708, 'sys': {'type': 1, 'id': 9588, 'message': 0.0073, 'country': 'AU', 'sunrise': 1561237936, 'sunset': 1561273121}, 'timezone': 36000, 'id': 2172517, 'name': 'City', 'cod': 200}]
[{'coord': {'lon': 149.13, 'lat': -35.28}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 1.13, 'pressure': 1027, 'humidity': 100, 'temp_min': -3.33, 'temp_max': 4.44}, 'visibility': 10000, 'wind': {'speed': 2.41, 'deg': 193.587}, 'clouds': {'all': 28}, 'dt': 1561235708, 'sys': {'type': 1, 'id': 9588, 'message': 0.0073, 'country': 'AU', 'sunrise': 1561237936, 'sunset': 1561273121},

[{'coord': {'lon': 149.13, 'lat': -35.28}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 1.13, 'pressure': 1027, 'humidity': 100, 'temp_min': -3.33, 'temp_max': 4.44}, 'visibility': 10000, 'wind': {'speed': 2.41, 'deg': 193.587}, 'clouds': {'all': 28}, 'dt': 1561235708, 'sys': {'type': 1, 'id': 9588, 'message': 0.0073, 'country': 'AU', 'sunrise': 1561237936, 'sunset': 1561273121}, 'timezone': 36000, 'id': 2172517, 'name': 'City', 'cod': 200}]
[{'coord': {'lon': 149.13, 'lat': -35.28}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 1.13, 'pressure': 1027, 'humidity': 100, 'temp_min': -3.33, 'temp_max': 4.44}, 'visibility': 10000, 'wind': {'speed': 2.41, 'deg': 193.587}, 'clouds': {'all': 28}, 'dt': 1561235708, 'sys': {'type': 1, 'id': 9588, 'message': 0.0073, 'country': 'AU', 'sunrise': 1561237936, 'sunset': 1561273121},

In [218]:
# Get current weather
get_city_data = owm.get_current("Hong kong", **settings)
print(f"Current weather object for Paris: {get_city_data}.")

Current weather object for Paris: {'coord': {'lon': 114.16, 'lat': 22.28}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}], 'base': 'stations', 'main': {'temp': 29.4, 'pressure': 1003, 'humidity': 74, 'temp_min': 28.33, 'temp_max': 30}, 'visibility': 10000, 'wind': {'speed': 6.2, 'deg': 200}, 'clouds': {'all': 20}, 'dt': 1561235613, 'sys': {'type': 1, 'id': 9154, 'message': 0.0082, 'country': 'CN', 'sunrise': 1561239628, 'sunset': 1561288217}, 'timezone': 28800, 'id': 1819729, 'name': 'Hong Kong', 'cod': 200}.


In [166]:
city = ["name"]
city_long = ["coord.lon"]
city_lat = ["coord.lat"]

city = get_city_data(*city)
city_long = get_city_data(*city_long)
city_lat = get_city_data(*city_lat)
# print(f"The current weather summary for Paris is: {data}.")

print(city)
print(city_long)
print(city_lat)

Hong Kong
114.16
22.28


In [ ]:
# Get data for each city in cities.csv
weather_data = []
with open("../Resources/cities.csv") as cities_file:
    cities_reader = csv.reader(cities_file)
    cities = [city[0] for city in cities_reader]
    weather_data = [ow.get_current(city, **settings) for city in cities]

In [ ]:
# Create an "extracts" object to get the temperature, latitude,
# and longitude in each city
summary = ["main.temp", "coord.lat", "coord.lon"]

# Create a Pandas DataFrame with the results
data = [response(*summary) for response in weather_data]

weather_data = pd.DataFrame(data, index=cities)
weather_data

In [ ]:

# cities_lat = [] 
# cities_long = []

# for index, row in cleanData.iterrows():
#     city_lat = results[0]['geometry']['lat']
#     city_long = results[0]['geometry']['lng']
    
#     rooms_lat.append(room_lat)
#     rooms_long.append(room_long)
#     cities_lat.append(city_lat)
#     cities_long.append(city_long)

In [ ]:
# #room_id: 19051656 / city: New-orleans, United states
# room_lat1 =  29.952861
# city_lat2 =  29.9499323
# room_long1 =  -90.071311
# city_long2 =  -90.0701156

# def distance(room_long1, room_lat1, city_long2, city_lat2):
#     room_long1, room_lat1, city_long2, city_lat2 = map(radians, [room_long1, room_lat1, city_long2, city_lat2])
#     dlon = city_long2 - room_long1
#     dlat = city_lat2 - room_lat1
#     a = sin(dlat / 2) ** 2 + cos(room_lat1) * cos(city_lat2) * sin(dlon / 2) ** 2
#     return 2 * 3958.756 * asin(sqrt(a)) #6371 km / 3958.756 for miles


In [63]:
# rooms_lat = []   # create empty lists
# rooms_long = []

# #Loops through each room long/lat and pushes to list
# for index, row in cleanData.iterrows():
#     room_lat = row['Latitude']
#     room_long = row['Longitude']

#     rooms_lat.append(room_lat)
#     rooms_long.append(room_long)
    
# # print(rooms_lat)
# # print(rooms_long)

In [124]:
# API_key = "30c5a732512c41af8f052a21f806fcce"

# geocoder = OpenCageGeocode(API_key)

# city_country = 'NYC, United states'

# results = geocoder.geocode(city_country)

# pprint (results)

In [65]:
# lat = results[0]['geometry']['lat']
# lng = results[0]['geometry']['lng']
# print (lat, lng)

In [125]:
# # this site confirmed the calculation is correct:
# # https://www.gpsvisualizer.com/calculators

# # OpenCageGeocode API call to get long lat of city name
# API_key = "30c5a732512c41af8f052a21f806fcce"
# geocoder = OpenCageGeocode(API_key)

# cities_lat = [] 
# cities_long = []

# for index, row in cleanData.iterrows():
    
#     results = geocoder.geocode(row['Selection'])
#     city_lat2 = results[0]['geometry']['lat']
#     city_long2 = results[0]['geometry']['lng']

#     cities_lat.append(city_lat)
#     cities_long.append(city_long)
    
# print(cities_lat)
# print(cities_long)

In [126]:
# for index, row in cleanData.iterrows():


In [66]:
# API_key = "30c5a732512c41af8f052a21f806fcce"

# geocoder = OpenCageGeocode(API_key)

# city_country = 'New-orleans, United states'

# results = geocoder.geocode(city_country)

# # print (results)

In [67]:
# API_key = "30c5a732512c41af8f052a21f806fcce"
# geocoder = OpenCageGeocode(API_key)

# cities_lat = []   # create empty lists
# cities_long = []

# for index, row in cleanData.iterrows():
# #     print(row['Selection'])
    
#     results = geocoder.geocode(row['Selection'])
#     city_lat = results[0]['geometry']['lat']
#     city_long = results[0]['geometry']['lng']
    
#     cities_lat.append(city_lat)
#     cities_long.append(city_long)
    
# # print(cities_lat)
# # print(cities_long)

## Lat/Long Distance Calculation

In [ ]:
# this site confirmed the calculation is correct:
# https://www.gpsvisualizer.com/calculators